<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Literature_match_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
Auto_5_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_185_TF_new.csv', error_bad_lines=False)
Auto_4_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_172_TF_new.csv', error_bad_lines=False)
Auto_3_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PUF_120k_167_TF_new.csv', error_bad_lines=False)
Auto_2_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PUF_120k_149_TF_new.csv', error_bad_lines=False)
Auto_1_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PUF_120k_138_TF_new.csv', error_bad_lines=False)

Auto_5_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_183_TF_new.csv', error_bad_lines=False)
Auto_4_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_179_TF_new.csv', error_bad_lines=False)
Auto_3_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PF_120k_170_TF_new.csv', error_bad_lines=False)
Auto_2_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PF_120k_147_TF_new.csv', error_bad_lines=False)
Auto_1_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PF_120k_125_TF_new.csv', error_bad_lines=False)

QC_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_PUF4_2_120k_159_TF_new.csv')

<ipython-input-2-717f992726da>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_5_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_185_TF_new.csv', error_bad_lines=False)
<ipython-input-2-717f992726da>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_4_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_172_TF_new.csv', error_bad_lines=False)
<ipython-input-2-717f992726da>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_3_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PUF_120k_167_TF_new.csv', error_bad_lines=False)
<ipython-input-2-717f992726da>:4: FutureWarning: The error_bad_lines argument has bee

In [3]:
Auto_5_gp_df['Ion Type'] = 'gp'
Auto_4_gp_df['Ion Type'] = 'gp'
Auto_3_gp_df['Ion Type'] = 'gp'
Auto_2_gp_df['Ion Type'] = 'gp'
Auto_1_gp_df['Ion Type'] = 'gp'

Auto_5_pm_df['Ion Type'] = 'pm'
Auto_4_pm_df['Ion Type'] = 'pm'
Auto_3_pm_df['Ion Type'] = 'pm'
Auto_2_pm_df['Ion Type'] = 'pm'
Auto_1_pm_df['Ion Type'] = 'pm'

Auto_5_gp_df['Sample Name'] = 'Auto_5'
Auto_4_gp_df['Sample Name'] = 'Auto_4'
Auto_3_gp_df['Sample Name'] = 'Auto_3'
Auto_2_gp_df['Sample Name'] = 'Auto_2'
Auto_1_gp_df['Sample Name'] = 'Auto_1'

Auto_5_pm_df['Sample Name'] = 'Auto_5'
Auto_4_pm_df['Sample Name'] = 'Auto_4'
Auto_3_pm_df['Sample Name'] = 'Auto_3'
Auto_2_pm_df['Sample Name'] = 'Auto_2'
Auto_1_pm_df['Sample Name'] = 'Auto_1'

In [4]:
print(Auto_3_gp_df.columns)

Index(['Component Name', 'Retention Time', 'Reference m/z', 'Area', 'Height',
       'TIC', 'Formula (mol ion)', 'CAS No.', 'SI', 'RSI', 'HRF Score',
       'RHRF Score', 'Total Score', 'Selected Column Type', 'Calculated RI',
       'Library RI', 'Delta RI', 'Library Name', 'Library ID Number',
       'Sample Name', 'File name', 'Ion Type'],
      dtype='object')


In [5]:
print('Auto5_gp:', Auto_5_gp_df.shape)
print('Auto4_gp:', Auto_4_gp_df.shape)
print('Auto5_pm:', Auto_5_pm_df.shape)
print('Auto4_pm:', Auto_4_pm_df.shape)

Auto5_gp: (583, 22)
Auto4_gp: (649, 22)
Auto5_pm: (269, 22)
Auto4_pm: (285, 22)


In [6]:
def area_ratio(df):
    """
    Calculates ratio of area to the internal standard and saves it in a new column
    """
    benzyl_benzoate_area = df.loc[df['Component Name'] == 'Benzyl Benzoate', 'Area'].values[0]
    df['Area Ratio'] = df['Area'] / benzyl_benzoate_area

    return df

def filter_dataframe(df):
    """
    Removing the results below the probability threshold that is set below
    """
    condition1 = df['RHRF Score'] >= 90
    condition2 = df['RSI'] >= 600
    condition3 = (df['Total Score'] >= 70) & ~(condition1 & condition2)

    filtered_df = df[condition1 & condition2 | condition3]

    return filtered_df

def rearrange_by_area_ratio(df):
    """
    Rearrearanging the rows in descending fashion from the peak area ratio
    """
    if 'Area Ratio' not in df.columns:
        raise ValueError("DataFrame must have a 'Area Ratio' column")

    df_sorted = df.sort_values(by='Area Ratio', ascending=False)

    #Reset the index to reflect the new order
    df_sorted = df_sorted.reset_index(drop=True)

    return df_sorted

import pandas as pd

def find_common_row(sample_df, ref_df):
    result_df = pd.DataFrame(columns=sample_df.columns)

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df)
        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)
            if best_match is not None and sample_row['Area Ratio'] >= (0.05 * best_match['Area Ratio']):
                result_df = result_df.append(sample_row, ignore_index=True)

    result_df = result_df[result_df['Component Name'].isin(ref_df['Component Name'])]

    return result_df

def qc_correction(sample_df, ref_df):
    result_df = sample_df.copy()
    matched_component_names = set()

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df)

        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)

            if best_match is not None:
                if sample_row['Area Ratio'] >= (0.05 * best_match['Area Ratio']):
                    matched_component_names.add(best_match['Component Name'])

    result_df = result_df[~result_df['Component Name'].isin(matched_component_names)]

    return result_df

def find_matching_rows(sample_row, ref_df):
    matching_rows = ref_df[ref_df['Formula (mol ion)'] == sample_row['Formula (mol ion)']]
    return matching_rows

def find_best_match(sample_row, matching_rows):
    best_match = None
    min_diff = float('inf')

    for _, ref_row in matching_rows.iterrows():
        diff = compare_strings(sample_row['Component Name'], ref_row['Component Name'])
        if diff < min_diff:
            min_diff = diff
            best_match = ref_row

    return best_match

def compare_strings(str1, str2):
    return 0 if str1 == str2 else 1

In [7]:
Auto_5_gp_df = area_ratio(Auto_5_gp_df)
Auto_4_gp_df = area_ratio(Auto_4_gp_df)
Auto_3_gp_df = area_ratio(Auto_3_gp_df)
Auto_2_gp_df = area_ratio(Auto_2_gp_df)
Auto_1_gp_df = area_ratio(Auto_1_gp_df)

Auto_5_pm_df = area_ratio(Auto_5_pm_df)
Auto_4_pm_df = area_ratio(Auto_4_pm_df)
Auto_3_pm_df = area_ratio(Auto_3_pm_df)
Auto_2_pm_df = area_ratio(Auto_2_pm_df)
Auto_1_pm_df = area_ratio(Auto_1_pm_df)

QC_df = area_ratio(QC_df)
print(rearrange_by_area_ratio(QC_df))

               Component Name  Retention Time  Reference m/z        Area  \
0     2,4-Di-tert-butylphenol          13.724     191.142929  2209640031   
1             Benzyl Benzoate          18.606     105.033531  1045399686   
2               Benzothiazole          10.349     135.013733   879269818   
3    Butylated Hydroxytoluene          13.673     205.158676   380545626   
4            Cyclotetradecane          11.249      69.069885   284540957   
..                        ...             ...            ...         ...   
482               Peak@14.496          14.496     135.044128      190941   
483               Peak@13.621          13.621     192.187241      181360   
484               Peak@23.279          23.279      85.095688      173998   
485               Peak@22.283          22.283      85.095688      152261   
486               Peak@24.900          24.900     345.035828      135732   

        Height        TIC Formula (mol ion)   CAS No.     SI    RSI  ...  \
0    388091

In [8]:
Auto_5_gp_df_filtered = qc_correction(Auto_5_gp_df, QC_df)
Auto_4_gp_df_filtered = qc_correction(Auto_4_gp_df, QC_df)
Auto_3_gp_df_filtered = qc_correction(Auto_3_gp_df, QC_df)
Auto_2_gp_df_filtered = qc_correction(Auto_2_gp_df, QC_df)
Auto_1_gp_df_filtered = qc_correction(Auto_1_gp_df, QC_df)

Auto_5_pm_df_filtered = qc_correction(Auto_5_pm_df, QC_df)
Auto_4_pm_df_filtered = qc_correction(Auto_4_pm_df, QC_df)
Auto_3_pm_df_filtered = qc_correction(Auto_3_pm_df, QC_df)
Auto_2_pm_df_filtered = qc_correction(Auto_2_pm_df, QC_df)
Auto_1_pm_df_filtered = qc_correction(Auto_1_pm_df, QC_df)

print('Auto5_gp:', Auto_5_gp_df_filtered.shape)
print('Auto4_gp:', Auto_4_gp_df_filtered.shape)
print('Auto5_pm:', Auto_5_pm_df_filtered.shape)
print('Auto4_pm:', Auto_4_pm_df_filtered.shape)
#Auto_5_gp_df_filtered.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_corrected.csv')

Auto5_gp: (504, 23)
Auto4_gp: (560, 23)
Auto5_pm: (223, 23)
Auto4_pm: (239, 23)


In [9]:
Auto5_gp_comon_df = find_common_row(Auto_5_gp_df, QC_df)
Auto4_gp_comon_df = find_common_row(Auto_4_gp_df, QC_df)
Auto3_gp_comon_df = find_common_row(Auto_3_gp_df, QC_df)
Auto2_gp_comon_df = find_common_row(Auto_2_gp_df, QC_df)
Auto1_gp_comon_df = find_common_row(Auto_1_gp_df, QC_df)


Auto5_pm_comon_df = find_common_row(Auto_5_pm_df, QC_df)
Auto4_pm_comon_df = find_common_row(Auto_4_pm_df, QC_df)
Auto3_pm_comon_df = find_common_row(Auto_3_pm_df, QC_df)
Auto2_pm_comon_df = find_common_row(Auto_2_pm_df, QC_df)
Auto1_pm_comon_df = find_common_row(Auto_1_pm_df, QC_df)

print('Auto5_gp:', Auto5_gp_comon_df.shape)
print('Auto4_gp:', Auto4_gp_comon_df.shape)
print('Auto5_pm:', Auto5_pm_comon_df.shape)
print('Auto4_pm:', Auto4_pm_comon_df.shape)
#print(rearrange_by_area_ratio(Auto5_gp_comon_df))
#Auto5_gp_comon_df.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_matches.csv')

<ipython-input-6-00ec0475d96e>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-6-00ec0475d96e>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-6-00ec0475d96e>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-6-00ec0475d96e>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-6-00ec0475d96e>:46: FutureWarning: The frame.append method is dep

Auto5_gp: (79, 23)
Auto4_gp: (88, 23)
Auto5_pm: (46, 23)
Auto4_pm: (46, 23)


In [10]:
Auto5_gp_ordered = rearrange_by_area_ratio(Auto_5_gp_df_filtered)
Auto4_gp_ordered = rearrange_by_area_ratio(Auto_4_gp_df_filtered)
Auto3_gp_ordered = rearrange_by_area_ratio(Auto_3_gp_df_filtered)
Auto2_gp_ordered = rearrange_by_area_ratio(Auto_2_gp_df_filtered)
Auto1_gp_ordered = rearrange_by_area_ratio(Auto_1_gp_df_filtered)


Auto5_pm_ordered = rearrange_by_area_ratio(Auto_5_pm_df_filtered)
Auto4_pm_ordered = rearrange_by_area_ratio(Auto_4_pm_df_filtered)
Auto3_pm_ordered = rearrange_by_area_ratio(Auto_3_pm_df_filtered)
Auto2_pm_ordered = rearrange_by_area_ratio(Auto_2_pm_df_filtered)
Auto1_pm_ordered = rearrange_by_area_ratio(Auto_1_pm_df_filtered)
print(Auto5_gp_ordered.head())
print(Auto4_gp_ordered.head())
print(Auto5_pm_ordered.head())
print(Auto4_pm_ordered.head())

                                      Component Name  Retention Time  \
0  Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
1              Carbonic acid, tetradecyl vinyl ester           9.683   
2                          Octane, 5-ethyl-2-methyl-           6.708   
3                 Carbonic acid, decyl undecyl ester          11.236   
4  1-(2-Methoxyethoxy)-2-methyl-2-propanol, methy...           5.970   

   Reference m/z        Area     Height         TIC Formula (mol ion)  \
0     219.174301  3487699203  201822395  2227097584          C35H62O3   
1      43.054165  2929291121  479368820  2731429243          C17H32O3   
2      43.054165  1872334548  296600545  1156799972            C11H24   
3      43.054165  1854075664  323340139  2471879501          C22H44O3   
4      73.064796  1649548412  278284446   507542018           C8H18O3   

      CAS No.     SI    RSI  ...  Selected Column Type  Calculated RI  \
0   2082-79-3  824.0  831.0  ...      StandardNonPolar 

In [11]:
Autp5_gp_final = filter_dataframe(Auto_5_gp_df_filtered)
Autp4_gp_final = filter_dataframe(Auto_4_gp_df_filtered)
Autp3_gp_final = filter_dataframe(Auto_3_gp_df_filtered)
Autp2_gp_final = filter_dataframe(Auto_2_gp_df_filtered)
Autp1_gp_final = filter_dataframe(Auto_1_gp_df_filtered)


Autp5_pm_final = filter_dataframe(Auto_5_pm_df_filtered)
Autp4_pm_final = filter_dataframe(Auto_4_pm_df_filtered)
Autp3_pm_final = filter_dataframe(Auto_3_pm_df_filtered)
Autp2_pm_final = filter_dataframe(Auto_2_pm_df_filtered)
Autp1_pm_final = filter_dataframe(Auto_1_pm_df_filtered)

print('Auto5_gp:', Autp5_gp_final.shape)
print('Auto4_gp:', Autp4_gp_final.shape)
print('Auto5_pm:', Autp5_pm_final.shape)
print('Auto4_pm:', Autp4_pm_final.shape)
Autp5_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_filtered.csv')
Autp4_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_blank_filtered.csv')
Autp3_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PUF_120k_blank_filtered.csv')
Autp2_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PUF_120k_blank_filtered.csv')
Autp1_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PUF_120k_blank_filtered.csv')

Autp5_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_blank_filtered.csv')
Autp4_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_blank_filtered.csv')
Autp3_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto3_PF_120k_blank_filtered.csv')
Autp2_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto2_PF_120k_blank_filtered.csv')
Autp1_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto1_PF_120k_blank_filtered.csv')


Auto5_gp: (258, 23)
Auto4_gp: (257, 23)
Auto5_pm: (93, 23)
Auto4_pm: (92, 23)


In [12]:
Autp5_gp_final_ordered = rearrange_by_area_ratio(Autp5_gp_final)
Autp4_gp_final_ordered = rearrange_by_area_ratio(Autp4_gp_final)
Autp5_pm_final_orderde = rearrange_by_area_ratio(Autp5_pm_final)
Autp4_pm_final_ordered = rearrange_by_area_ratio(Autp4_pm_final)
print(Autp5_gp_final_ordered.head())
print(Autp4_gp_final_ordered.head())
print(Autp5_pm_final_orderde.head())
print(Autp4_pm_final_ordered.head())

                                      Component Name  Retention Time  \
0  Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
1              Carbonic acid, tetradecyl vinyl ester           9.683   
2  1-(2-Methoxyethoxy)-2-methyl-2-propanol, methy...           5.970   
3                                1-Decanol, 2-ethyl-           9.092   
4                                 2,3-Dimethyldecane           8.152   

   Reference m/z        Area     Height         TIC Formula (mol ion)  \
0     219.174301  3487699203  201822395  2227097584          C35H62O3   
1      43.054165  2929291121  479368820  2731429243          C17H32O3   
2      73.064796  1649548412  278284446   507542018           C8H18O3   
3      43.054165  1625300363  196613969  1242090279           C12H26O   
4      43.054165  1543692479  272134554  1410395910            C12H26   

      CAS No.     SI    RSI  ...  Selected Column Type  Calculated RI  \
0   2082-79-3  824.0  831.0  ...      StandardNonPolar 